In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
from nltk.tag.perceptron import PerceptronTagger
tagger = PerceptronTagger()

Example on how to classify a sentece. Here the split() function splits by ' ' character.

Create a token class

In [ ]:
class Token:
  spelling = None
  POSTag = None

  def __init__ (self, spelling, tag):
    self.spelling = spelling.lower().strip()
    self.POSTag = tag.strip()


In [ ]:
class Sentence:
  sentence = None

  def __init__ (self, ListOfTokens):
    self.sentence = ListOfTokens

  def ToString(self):
    sentenceString = ""
    for token in self.sentence:
      sentenceString += str(token.spelling) + " "
    return sentenceString


In [ ]:
class DataSet:
  dataSet = None

  def __init__ (self, ListOfSentences):
    self.dataSet = ListOfSentences

  def split(self, fraction):
    dataSetSize = len(self.dataSet)
    indexOfSplit = int(fraction * dataSetSize)

    trainingDataSet = DataSet(self.dataSet[:indexOfSplit])
    testingDataSet = DataSet(self.dataSet[indexOfSplit:])

    return trainingDataSet, testingDataSet


2.)
- Upload the files
- Re-format the sentences
- All words to lower case



In [ ]:
from google.colab import files
uploaded = files.upload()

Saving BrownCorpus.txt to BrownCorpus.txt
Saving BrownToUniversalTagMap.txt to BrownToUniversalTagMap.txt


In [ ]:
completeDataSetData = None

with open("BrownCorpus.txt") as file:
  completeDataSetData = file.readlines()
completeDataSetData = np.array(completeDataSetData)

print("File completly read. " + str(completeDataSetData.size) + " senteces loaded.")

print("Convert the sentences to tokens, senteces and a dataSet")

tokenList = []
sentenceList = []

# loop over each element in the completeDataSet
for sentenceData in completeDataSetData:
  # split the sentece in tokens, use space as delimiter
  splitSentence = sentenceData.split()
  sentenceToken = []
  for subToken in splitSentence:
    splitToken = subToken.split("_")
    spelling = splitToken[0]
    if ('|' not in spelling or len(splitToken[1]) == 1):
      token = Token(splitToken[0], splitToken[1])
      tokenList.append(token)
      sentenceToken.append(token)
  sentence = Sentence(sentenceToken)
  sentenceList.append(sentence)

# initiate the DataSet
completeDataSet = DataSet(sentenceList)

# Convert all tags from Brown to Universal
BrownToUniversalTagData = []

with open("BrownToUniversalTagMap.txt") as file:
  for line in file:
    # replace \t\t with \t
    processedLine = line.replace("\t\t", "\t")
    processedLine = processedLine.split('\t')
    BrownTag = processedLine[0]
    UniversalTag = processedLine[1].strip()
    BrownToUniversalTagData.append([BrownTag, UniversalTag])


# convert to Numpy Array
BrownToUniversalTagData = np.array(BrownToUniversalTagData)

# Convert the BrownTags of the Tokens into Universal Tags
for token in tokenList:
  tokenBrownTag = token.POSTag
  if tokenBrownTag == "":
    print(token.Spelling)
  tokenBrownTagIndex = np.where(BrownToUniversalTagData[:, 0] == tokenBrownTag)

  tokenUniversalTag = BrownToUniversalTagData[tokenBrownTagIndex, 1][0][0]
  # get the index of
  #try:
  #except:
  ##  tokenUniversalTag = BrownToUniversalTagData[tokenBrownTagIndex, 1][0][0]
    #print(token.spelling)
    #print(token.POSTag)
  token.POSTag = tokenUniversalTag

print("Conversion of Brown to Universal tag for all tokens done!")

File completly read. 57066 senteces loaded.
Convert the sentences to tokens, senteces and a dataSet


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
import random
randS = random.randint(0, len(sentenceList))
sentence = sentenceList[randS]
randT = random.randint(0, len(sentence.sentence))

print(sentence.ToString())
print("Spelling:"+ '\t' + sentence.sentence[randT].spelling)
print("Tag:" + '\t'+ sentence.sentence[randT].POSTag)


a site may also be attractive just through the beauty of its trees and shrubs . 
Spelling:	its
Tag:	DET


3.)
- Split the data into a training and test set -> Exacetly like in part a) to make comparison

In [ ]:
## Split the data set by a fraction
fraction = 0.8
trainingDataSet, testingDataSet = completeDataSet.split(fraction)

print(len(completeDataSet.dataSet))
print(len(trainingDataSet.dataSet))
print(len(testingDataSet.dataSet))


57066
45652
11414


4.)
- Tokenize the test
--> Already done, with the implementation above

5.)
- Run the perceptron tagger over the test set

--> Classify every token in the test set

In [ ]:
resultsWithAssignedTags = []
from nltk.tag.mapping import map_tag
nltk.download('universal_tagset')

# loop over each sentence
for sentence in testingDataSet.dataSet:
  for token in sentence.sentence:
    spelling = str(token.spelling)

    trueTag = token.POSTag

    assignedTag = tagger.tag(spelling.split())[0][1]

    mappedTag = map_tag('en-ptb', 'universal', assignedTag)

    success = int(mappedTag == trueTag)

    results = [spelling, trueTag, mappedTag, success]
    resultsWithAssignedTags.append(results)

resultsWithAssignedTags = np.array(resultsWithAssignedTags)


[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


7.)
-> Compute the accuracy of the perceptron tagger.

In [ ]:
# take sum of all values in the 4th column of the resultsWithAssignedTags array
successSum = 0
numberOfEntries = resultsWithAssignedTags.shape[0]

for i in range(numberOfEntries):
  successSum += int(resultsWithAssignedTags[i, 3])

accuracy = successSum / numberOfEntries
print("Accuracy of tagger:\t\t", "{:.3%}".format(accuracy))
print("Number of tokens tagged:\t", numberOfEntries)


Accuracy of tagger:		 82.413%
Number of tokens tagged:	 176041


In [ ]:
# Initialize POS tag counters
pos_tag_counters = {
    '.' : 0,
    'ADJ': 0,
    'ADP': 0,
    'ADV': 0,
    'CONJ': 0,
    'DET': 0,
    'NOUN': 0,
    'NUM': 0,
    'PRON': 0,
    'PRT': 0,
    'VERB': 0,
    'X': 0,
    'BLANKS': 0
}

for token in tokenList:
  tag = token.POSTag
  # Increment POS tag counters
  if tag in pos_tag_counters:
      pos_tag_counters[tag] += 1
  else:
      pos_tag_counters['BLANKS'] += 1


# Sort the pos_tag_counters dictionary by tag
sorted_counters = dict(sorted(pos_tag_counters.items(), key=lambda x: x[1], reverse=True))

# Generate basic statistics -> Check if same frequencies
# Print POS tag counts and the fractions
fractionSum = 0
print("\nPOS Tag Counts:")
print("{:<10} {:<15} {:<15}".format("Tag", "Count", "Fraction"))
print("="*40)

for tag, count in sorted_counters.items():
    fraction = count / len(tokenList)
    fractionSum += fraction
    print("{:<10} {:<15} {:<15.4%}".format(tag, count, fraction))

print("="*40)
print(f"Total Fraction: {fractionSum:.4%}")


POS Tag Counts:
Tag        Count           Fraction       
NOUN       275723          24.2401%       
VERB       182905          16.0800%       
ADP        144848          12.7343%       
DET        137094          12.0526%       
.          123252          10.8357%       
ADJ        83766           7.3643%        
ADV        56282           4.9480%        
PRON       49389           4.3420%        
CONJ       38163           3.3551%        
PRT        29862           2.6253%        
NUM        14952           1.3145%        
X          1230            0.1081%        
BLANKS     0               0.0000%        
Total Fraction: 100.0000%
